# Tool usage in LLMs

LLMs can leverage tools in many ways such as

- Functions within LLM frameworks (Langchain, LangGraphDSPy)
- Functions registered with Unity Catalog (sql, python)
- Tools with external tools (REST APIs, etc)
- Databricks managed MCP Servers 
- External MCP Servers (can be hosted within Databricks)


In [0]:
%pip install databricks-sdk==0.41.0 
dbutils.library.restartPython()

In [0]:
# %run ./_resources/00-init-stylist $reset_all=false
catalog = "users"
dbName = db = "rob_bajra"
volume_name = "dbdemos_agent_volume"

spark.sql(f"USE `{catalog}`.`{db}`")

## Example 1: Unity Catalog sql function


In [0]:
%sql
CREATE OR REPLACE FUNCTION convert_inch_to_cm(size_in_inch FLOAT)
RETURNS FLOAT
LANGUAGE SQL
COMMENT 'convert size from inch to cm'
RETURN size_in_inch * 2.54;

-- let's test our function:
SELECT convert_inch_to_cm(10) as 10_inches_in_cm;

## Example 2: Unity Catalog python function

In [0]:
# %sql
# CREATE OR REPLACE FUNCTION get_customer_orders ()
# RETURNS TABLE(user_id STRING,
#   id STRING,
#   transaction_date STRING,
#   item_count DOUBLE,
#   amount DOUBLE,
#   order_status STRING)
# COMMENT 'Returns a list of customer orders for the given customer ID (expect a UUID)'
# LANGUAGE SQL
#     RETURN
#     SELECT o.* from tools_orders o 
#     inner join tools_customers c on c.id = o.user_id 
#     where email=current_user() ORDER BY transaction_date desc;

# SELECT * FROM get_customer_orders();

## Example 3: Unity Catalog external API function


In [0]:
%sql
CREATE OR REPLACE FUNCTION get_country_detail(country_name STRING)
RETURNS STRUCT<country_details STRING>
LANGUAGE PYTHON
COMMENT 'This function retrieves the details about a given country. Returns details about country flag, languages, etc' 
AS
$$
  try:
    import requests as r
    import json
    url = f'https://restcountries.com/v3.1/name/{country_name}/'
    response = r.get(url)

    return {
      "country_details": json.dumps(response.json())
    }
  except:
    return {"country_details": "Country not known"}
$$;

-- let's test our function:
SELECT get_country_detail("USA") as dets;



## Example 4: Creating a function calling LLMs with specific prompt as a tool

You can also register tools containing custom prompts that your LLM can use to to execute actions based on the customer context.

Let's create a tool that recommend the style for our user, based on the current weather.

In [0]:
%sql
CREATE OR REPLACE FUNCTION recommend_outfit_description(requested_style STRING, temperature_in_celsius FLOAT, rain_in_mm FLOAT)
RETURNS STRING
LANGUAGE SQL
COMMENT 'This function generate a stylist outfit description based on initial request and the current weather.'
RETURN SELECT ai_query('databricks-meta-llama-3-3-70b-instruct',
    CONCAT("You are a stylist assistant. Your goal is to give recommendation on what would be the best outfit for today. The current temperature is ",temperature_in_celsius ," celsius and rain is:", rain_in_mm, "mm. Give size in inches if any. Don't assume customer size if they don't share it. Give ideas of colors and other items to match. The user added this instruction based on what they like: ", requested_style)
  ) AS recommended_outfit;

-- let's test our function:
SELECT recommend_outfit_description("I need a dress for an interview.", 30.1, 0.0)

In [0]:
%sql
CREATE OR REPLACE FUNCTION compute_math(expr STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Run any mathematical function and returns the result as output. Supports python syntax like math.sqrt(13)'
AS
$$
  import ast
  import operator
  import math
  operators = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv, ast.Pow: operator.pow, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv, ast.UAdd: operator.pos, ast.USub: operator.neg}
    
  # Supported functions from the math module
  functions = {name: getattr(math, name) for name in dir(math) if callable(getattr(math, name))}

  def eval_node(node):
    if isinstance(node, ast.Num):  # <number>
      return node.n
    elif isinstance(node, ast.BinOp):  # <left> <operator> <right>
      return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
    elif isinstance(node, ast.UnaryOp):  # <operator> <operand> e.g., -1
      return operators[type(node.op)](eval_node(node.operand))
    elif isinstance(node, ast.Call):  # <func>(<args>)
      func = node.func.id
      if func in functions:
        args = [eval_node(arg) for arg in node.args]
        return functions[func](*args)
      else:
        raise TypeError(f"Unsupported function: {func}")
    else:
      raise TypeError(f"Unsupported type: {type(node)}")  
  try:
    if expr.startswith('```') and expr.endswith('```'):
      expr = expr[3:-3].strip()      
    node = ast.parse(expr, mode='eval').body
    return eval_node(node)
  except Exception as ex:
    return str(ex)
$$;
-- let's test our function:
SELECT compute_math("(2+2)/3") as result;

In [0]:
%sql
CREATE OR REPLACE FUNCTION execute_python_code(python_code STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT "Run python code. The code should end with a return statement and this function will return it as a string. Only send valid python to this function. Here is an exampe of python code input: 'def square_function(number):\\n  return number*number\\n\\nreturn square_function(3)'"
AS
$$
    import traceback
    try:
        import re
        # Remove code block markers (e.g., ```python) and strip whitespace```
        python_code = re.sub(r"^\s*```(?:python)?|```\s*$", "", python_code).strip()
        # Unescape any escaped newline characters
        python_code = python_code.replace("\\n", "\n")
        # Properly indent the code for wrapping
        indented_code = "\n    ".join(python_code.split("\n"))
        # Define a wrapper function to execute the code
        exec_globals = {}
        exec_locals = {}
        wrapper_code = "def _temp_function():\n    "+indented_code
        exec(wrapper_code, exec_globals, exec_locals)
        # Execute the wrapped function and return its output
        result = exec_locals["_temp_function"]()
        return result
    except Exception as ex:
        return traceback.format_exc()
$$;
-- let's test our function:

SELECT execute_python_code("return 'Hello World! '* 3") as result;